# <center>Ultralytics Pose Estimation</center>

Pose Estimation 是基于 YOLO 模型的扩展，专门用于姿态估计任务。它结合了 YOLO 的快速检测能力和关键点定位技术，能够高效地完成姿态估计任务。本节课将详细介绍 Ultralytics Pose Estimation 的结构特点、模型导出以及自定义插件的注册方法，帮助大家更好地理解和应用这一技术。

## <center>Ultralytics Pose Estimation 结构分析</center>
<!-- 
&emsp;&emsp;Ultralytics OBB 模型的核心优势在于其能够处理旋转的边界框。与 Ultralytics Detect 模型相比，[Ultralytics OBB 模型的输出在每个边界框中多了一个表示旋转角度的参数](https://github.com/ultralytics/ultralytics/blob/v8.3.23/ultralytics/nn/modules/head.py#L217)。 -->

In [ ]:
import netron
from IPython.display import IFrame, display

url, port = netron.start("./course_data/models/POSE/yolo11n-pose.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='520px'))

与 Detect 模型类似，Pose 模型的输入节点同样接收一个包含 N 个样本的批次，每个样本是一张具有 3 个颜色通道、尺寸为高 × 宽像素的图像。但在输出节点上存在一些差异。Pose 模型的输出维度为 `(batch, 4 + classes + kpts, predicts)`。其中，4 表示 anchor 坐标 `(x, y, w, h)`，classes 表示类别的置信度，kpts 代表关键点信息。关键点信息可以是 `(x, y)` 坐标，也可以是 `(x, y, visible)` 格式，具体取决于模型的配置。

例如，当输出维度为 56 时，可以拆解为：4（anchor 坐标）+ 1（类别置信度）+ 17 × 3（关键点信息，假设每个关键点有 3 个值：x、y 和可见性标志）。这种输出格式使得 Pose 模型能够同时提供目标检测和关键点定位的功能。

## <center>Ultralytics Pose Estimation 注册自定义插件</center>

在 YOLO Pose 模型中，为了提高非极大值抑制（NMS）后处理的效率，我们可以通过集成自定义插件来优化性能。由于标准的 EfficientNMS 插件不支持关键点信息的处理，为了解决这些问题，在之前的章节中我们开发了一个名为 EfficientIdxNMS 的自定义插件。相较于 EfficientNMS 插件，EfficientIdxNMS 多了一个 `detection_indices` 节点，用于获取筛选后的索引值，可以通过索引值获取到对应的键点信息，从而确保姿态估计的准确性。

In [1]:
import onnx
import onnxsim
import torch
from ultralytics import YOLO
from course_functions.head import UltralyticsPose

def export_model(model_path: str, output_path: str, batch: int = 1, 
                 imgsz: tuple = (640, 640), max_boxes: int = 100, 
                 iou_thres: float = 0.45, conf_thres: float = 0.25, 
                 opset_version: int = 11) -> None:
    """
    将 YOLO 模型导出为 ONNX 格式，并修改检测头。
    
    参数：
    - model_path: YOLO 模型权重文件的路径。
    - output_path: 导出的 ONNX 模型保存路径。
    - batch: 模型的批量大小，默认为 1。
    - imgsz: 模型输入的图像尺寸，默认为 (640, 640)。
    - max_boxes: 每张图片的最大检测数量，默认为 100。
    - iou_thres: NMS 的 IoU 阈值，默认为 0.25。
    - conf_thres: 检测的置信度阈值，默认为 0.25。
    - opset_version: ONNX opset 版本，默认为 11。
    """
    # 加载指定权重的 YOLO 模型并设置为 CPU 模式
    model = YOLO(model=model_path, verbose=False).model.to('cpu')
    img_h, img_w = imgsz

    # 修改 Detect 层的参数
    for m in model.modules():
        if m.__class__.__name__ == "Pose":
            m.__class__ = type("Pose", (UltralyticsPose,), {
                "dynamic": batch <= 0,  # 是否需要动态批量大小
                "max_det": max_boxes,  # 每张图片的最大检测数量
                "iou_thres": iou_thres,  # NMS 的 IoU 阈值
                "conf_thres": conf_thres,  # 检测的置信度阈值
            })
            break

    # 创建模型的虚拟输入张量
    dummy_input = torch.randn(1 if batch <= 0 else batch, 3, *imgsz).to('cpu')
    model.eval().float()  # 将模型设置为评估模式，并确保其为浮点精度
    dummy_output = model(dummy_input)  # 使用虚拟输入运行模型，以确保其正确配置
    
    # 将模型导出为 ONNX 格式
    torch.onnx.export(
        model,
        dummy_input,
        output_path,
        export_params=True,
        opset_version=16,
        do_constant_folding=True,
        input_names=['images'],
        output_names=["num_dets", "det_boxes", "det_scores", "det_classes", "det_kpts"],
        dynamic_axes={
            "images": {0: "batch", 2: "height", 3: "width"},
            "num_dets": {0: "batch"},
            "det_boxes": {0: "batch"},
            "det_scores": {0: "batch"},
            "det_classes": {0: "batch"},
            "det_kpts": {0: "batch"},
        } if batch <= 0 else None,  # 如果批量大小是动态的，则定义动态轴
    )

    # 加载导出的 ONNX 模型并进行验证
    model_onnx = onnx.load(output_path)
    onnx.checker.check_model(model_onnx)

    # 则更新输出节点维度
    shapes = {
        'num_dets': ["batch" if batch <= 0 else batch, 1],
        'det_boxes': ["batch" if batch <= 0 else batch, max_boxes, 4],
        'det_scores': ["batch" if batch <= 0 else batch, max_boxes],
        'det_classes': ["batch" if batch <= 0 else batch, max_boxes],
        'det_kpts': ["batch" if batch <= 0 else batch, max_boxes, dummy_output[-1].shape[-2], dummy_output[-1].shape[-1]],
    }
    for node in model_onnx.graph.output:
        for idx, dim in enumerate(node.type.tensor_type.shape.dim):
            dim.dim_param = str(shapes[node.name][idx])

    # 简化 ONNX 模型
    model_onnx, check = onnxsim.simplify(model_onnx)
    assert check, "Simplified ONNX model could not be validated"
    onnx.save(model_onnx, output_path)  # 保存简化后的 ONNX 模型

# 调用函数导出模型
export_model("./course_data/models/POSE/yolo11n-pose.pt", "./course_data/models/POSE/yolo11n-pose_with_plugin.onnx")

/home/my_ubuntu/AI_deployment/TRT-TritonServer-Lesson/course_functions/head.py:333: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.shape != shape:
[W707 22:22:03.066928708 shape_type_inference.cpp:1995] Warning: The shape inference of TRT::EfficientIdxNMS_TRT type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W707 22:22:03.071885301 shape_type_inference.cpp:1995] Warning: The shape inference of TRT::EfficientIdxNMS_TRT type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W707 22:22:03.073169055 shape_type_inference.cpp:1995] Warning: The 

In [ ]:
url, port = netron.start("./course_data/models/POSE/yolo11n-pose_with_plugin.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='620px'))

## <center>总结</center>

本节课程我们深入探讨了 Ultralytics Pose Estimation 的结构特点。我们学习了如何将 YOLO Pose 模型导出为 ONNX 格式，并注册自定义插件以优化非极大值抑制（NMS）的效率。通过实践，我们掌握了如何修改 YOLO 模型的检测头，并成功导出了集成自定义插件的 ONNX 模型。这不仅提高了模型的推理速度，还为后续的部署提供了便利。